Import Libraries/Data

In [1]:
import os
import sys
import glob
import numpy as np
import torch
from torchvision import transforms, datasets
from PIL import Image

#Import image loaders
os.chdir("../../src/")
from load_data import preview_images, load_images
from models import train_loop, test_loop

In [2]:
#Import and preview image data
train_path= "../Data/train/"
horse_train= load_images(train_path + "horses/")
human_train= load_images(train_path + "humans/")

val_path= "../Data/validation/"
horse_test= load_images(val_path + "horses/")
human_test= load_images(val_path + "humans/")

#Sanity Check
print("Training Horse Data: %s \nTest Horse Data: %s\n" % (horse_train.shape, horse_test.shape))
print("Training Human Data: %s \nTest Human Data: %s" % (human_train.shape, human_test.shape))

Training Horse Data: (500, 300, 300, 4) 
Test Horse Data: (128, 300, 300, 4)

Training Human Data: (527, 300, 300, 4) 
Test Human Data: (128, 300, 300, 4)


Preprocess **Training** Data

In [3]:
#Assign labels: 1= human, 0 = horse
y_train_human= np.full((len(human_train),1), 1)
y_train_horse= np.full((len(horse_train),1), 0)

#Concatenate Training data
X= np.concatenate((horse_train, human_train), axis=0)
y= np.concatenate((y_train_horse, y_train_human), axis= 0)

#Zero-center data
training_mean= X.mean()
training_std= X.std()
X= (X - training_mean)/training_std

#Shuffle, reshape and convert to tensors
from sklearn.utils import shuffle
X_train, y_train= shuffle(X, y)

X_train= X_train.reshape((-1, 4, 300, 300))

X_train= torch.from_numpy(X_train)
y_train= torch.from_numpy(y_train)

#Sanity Check
print(X_train.shape)
print(y_train.shape)
#Wrap tensors into Dataset 
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)    

torch.Size([1027, 4, 300, 300])
torch.Size([1027, 1])


In [4]:
#Preprocess Test Data
y_test_human= np.full((len(human_test),1), 1)
y_test_horse= np.full((len(horse_test),1), 0)

#Concatenate validation set
X= np.concatenate((horse_test, human_test), axis=0)
y= np.concatenate((y_test_horse, y_test_human), axis= 0)

#Zero-Center data
X= (X- training_mean)/training_std

#Shuffle, reshape, and convert test data to tensors
X_test, y_test= shuffle(X, y)

X_test= X_test.reshape((-1, 4, 300, 300))

X_test= torch.from_numpy(X_test)
y_test= torch.from_numpy(y_test)

#Sanity Check
print("Test data shape: %s \nTest label shape: %s" % (X_test.shape, y_test.shape))

#Wrap tensors into Dataset 
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

Test data shape: torch.Size([256, 4, 300, 300]) 
Test label shape: torch.Size([256, 1])


# Modeling

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__() #Init parent class nn.Module
        self.conv1= nn.Conv2d(in_channels=3,out_channels= 32,kernel_size= 4, stride=1)
        self.conv2= nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride= 1)
        
        self.pool= nn.MaxPool2d(kernel_size= 3, stride= 1, padding=0 )
        self.fc1= nn.Linear(in_features= 64, out_features=256)
        self.fc2= nn.Linear(in_features= 256, out_features= 2)
        
    def forward(self, x):
        x= F.relu(self.conv1(x))
        x= self.pool(F.relu(self.conv2(x)))
        x= torch.flatten(x, 1)
        x= self.fc1(x)
        x= self.fc2(x)
        return x

        
        
        

In [7]:
conv_net= ConvNet()


In [13]:
batch_size= 64
criterion= nn.CrossEntropyLoss()

from torch import optim

optimizer= optim.SGD(conv_net.parameters(), lr= 10**-3, momentum=.9)

In [10]:
#create data loaders
from torch.utils.data import DataLoader

train_dataloader= DataLoader(train_dataset, batch_size= batch_size, drop_last= True)
test_dataloader= DataLoader(test_dataset, batch_size= len(test_dataset))

In [ ]:
epochs= 2
for e in epochs:
    train_loop(train_dataloader, conv_net, 